In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [112]:
df = pd.read_csv("SpeedDatingData.csv", encoding="ISO-8859-1")

In [209]:
def get_imputed_table(df, seed=0):

    rng = np.random.default_rng(seed=seed)

    df['scored_decision'] = df['like'] + df['dec']*0.5
    df['stdzed_scored_decision'] = df.groupby('iid')['scored_decision'].transform(lambda x: ((x - x.mean()) / x.std()).fillna(0))

    large_pivot = df[['iid', 'pid', 'stdzed_scored_decision']].pivot(index='iid', columns='pid', values='stdzed_scored_decision').iloc[:, 1:]
    randomly_generated = pd.DataFrame(rng.normal(0,1, size=large_pivot.shape), large_pivot.index, large_pivot.columns)
    imputed_score = large_pivot.fillna(randomly_generated)

    return imputed_score

In [210]:
imputed_table = get_imputed_table(df)

In [176]:
gender_0 = df[df['gender']==0]['iid'].unique()
gender_1 = df[df['gender']==1]['iid'].unique()

In [177]:
preferences_0 = imputed_table.loc[gender_0, gender_1]
preferences_1 = imputed_table.loc[gender_1, gender_0]

In [212]:
preference_matrix = pd.DataFrame(np.tril(imputed_table.to_numpy()) + np.tril(imputed_table.to_numpy().T))

In [216]:
RHO = -0.5
np.fill_diagonal(preference_matrix.values, RHO)

In [278]:
def get_heuristic_pairs(ld_preference_matrix, rho=RHO):
    used = set()
    total_value = {}
    for i in ld_preference_matrix:
        if i not in used:
            ordered_column = ld_preference_matrix.iloc[:, i].sort_values(ascending=False)
            avail = ordered_column[(ordered_column >= rho) & ~(ordered_column.index.isin(used))]            
            used.add(avail.index[0])
            used.add(i)
            total_value[(i, avail.index[0])] = avail.iloc[0]

    return used, total_value

In [279]:
used,total_value = get_heuristic_pairs(preference_matrix)

In [289]:
total_value.values()

dict_values([4.626397786381151, 4.00418418788827, 3.8034066488664795, 4.076051862280507, 4.432307254016068, 4.118383168787993, 4.14406984192828, 4.760705842841834, 4.298032908817408, 4.991396488099523, 4.917953502425624, 3.4772216835053915, 5.011228406724456, 4.766365464258271, 4.56019759808465, 3.70409620913496, 3.770958344461807, 4.590675553106294, 5.177716283968772, 4.24840337020105, 3.8345642360676067, 3.668693462905319, 4.145112982855286, 4.494822045486998, 4.144166032956564, 4.250037748351817, 4.924576645385508, 3.8739243269304486, 3.922726317020719, 4.073776219233168, 4.423228525653205, 3.596389661120061, 5.385768919377853, 4.539409604847145, 5.152546481311649, 4.7198188712246685, 5.234780100482425, 4.017762934430214, 4.948525460719459, 5.20255205946747, 4.833265718972056, 3.905436779039877, 4.082509196505471, 4.92774224834112, 4.115315077705365, 4.419975520475395, 3.639702298211243, 4.3542795818071305, 4.491456210303221, 4.798000060108688, 4.741570613280686, 5.2553825706913635,

In [266]:
for i in range(551):
    print(i, i in np.concatenate(list(total_value.keys())))

0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True
19 True
20 True
21 True
22 True
23 True
24 True
25 True
26 True
27 True
28 True
29 True
30 True
31 True
32 True
33 True
34 True
35 True
36 True
37 True
38 True
39 True
40 True
41 True
42 True
43 True
44 True
45 True
46 True
47 True
48 True
49 True
50 True
51 True
52 True
53 True
54 True
55 True
56 True
57 True
58 True
59 True
60 True
61 True
62 True
63 True
64 True
65 True
66 True
67 True
68 True
69 True
70 True
71 True
72 True
73 True
74 True
75 True
76 True
77 True
78 True
79 True
80 True
81 True
82 True
83 True
84 True
85 True
86 True
87 True
88 True
89 True
90 True
91 True
92 True
93 True
94 True
95 True
96 True
97 True
98 True
99 True
100 True
101 True
102 True
103 True
104 True
105 True
106 True
107 True
108 True
109 True
110 True
111 True
112 True
113 True
114 True
115 True
116 True
117 True
118 True
119 True
120 True
121 True
122 True
123

In [200]:
preference_matrix[0].sort_values().iloc[0]

-4.503608697651545

In [73]:
optimal_pairs = {}
total_value = {}
for i in preference_table.columns:
    ordered_column = preference_table[i].nlargest(20)
    remaining = ordered_column[(ordered_column >-1) & (~ordered_column.index.isin(optimal_pairs.values()))]
    if len(remaining) > 0:
        optimal_pairs[i] = remaining.idxmax()
        total_value[(i, optimal_pairs[i])] = ordered_column[optimal_pairs[i]]
    else:
        optimal_pairs[i] = 0
        total_value[(i, optimal_pairs[i])] = -0.5

In [6]:
wave9 = df[df['wave']==9][['iid', 'gender', 'pid', 'stdzed_scored_decision']]

In [7]:
preferences_0 = wave9[wave9['gender']==0].pivot(index='iid', columns='pid', values='stdzed_scored_decision').fillna(0)
preferences_1 = wave9[wave9['gender']==1].pivot(index='iid', columns='pid', values='stdzed_scored_decision').fillna(0)
#preferences_0.to_csv('preferences_0.csv')
#preferences_1.to_csv('preferences_1.csv')

In [149]:
preferences_0

pid,214.0,215.0,216.0,217.0,218.0,219.0,220.0,221.0,222.0,223.0,224.0,225.0,226.0,227.0,228.0,229.0,230.0,231.0,232.0,233.0
iid,,,,,,,,,,,,,,,,,,,,
194,0.700783,1.605020,-1.559808,-0.203453,-0.203453,-0.203453,0.248665,1.605020,-1.559808,-0.655572,0.700783,0.248665,0.700783,0.700783,-0.655572,0.700783,-1.559808,0.700783,-1.559808,0.248665
195,1.596872,1.909984,-0.908025,-0.594913,-0.281801,0.970648,0.970648,-0.281801,-1.534250,0.344423,0.344423,0.657536,0.031311,-0.281801,-0.908025,0.031311,-1.534250,0.970648,-1.534250,0.031311
196,1.238009,1.238009,1.463102,2.138379,-0.787824,1.688194,-0.787824,-0.337639,-0.787824,-0.337639,-1.238009,-0.787824,-0.337639,-0.337639,-1.238009,-0.337639,-0.337639,0.112546,-0.337639,0.112546
197,-0.579409,-0.579409,-0.579409,1.146489,-0.579409,-0.579409,-0.579409,-0.579409,-0.579409,-0.086295,-0.086295,-0.579409,0.406819,-0.086295,-0.086295,-0.579409,-0.579409,2.625830,2.625830,-0.086295
198,-0.295561,1.182244,0.443342,-0.295561,-0.295561,-0.295561,-1.773366,-1.034464,-1.773366,-0.295561,-0.295561,0.443342,1.182244,1.182244,1.182244,1.182244,0.443342,0.443342,-1.773366,0.443342
199,0.385185,0.977777,-1.195061,0.187654,0.187654,1.372838,0.780246,0.977777,-1.195061,1.175308,-0.009877,0.977777,0.977777,0.385185,-0.009877,-1.195061,-1.590122,-1.195061,-1.590122,-0.404938
200,0.860309,0.860309,-1.755030,1.514144,-0.447361,0.860309,0.860309,-0.447361,-0.447361,0.860309,1.514144,0.860309,-0.447361,-0.447361,-0.447361,-1.101196,-1.755030,-0.447361,-0.447361,0.000000
201,0.707511,0.707511,0.173540,0.440526,-2.496313,-0.894401,-0.894401,0.173540,-1.962342,0.707511,-0.360430,0.173540,1.508467,0.173540,-0.360430,0.707511,1.508467,0.173540,-0.360430,0.173540
202,1.086515,1.086515,-0.585047,1.504405,-0.585047,0.250734,-1.420827,-0.585047,-1.420827,0.250734,-0.585047,0.250734,1.504405,0.250734,-1.420827,1.504405,-1.420827,0.250734,0.250734,-0.167156


In [8]:
preference_table = preferences_0 + preferences_1.T

In [77]:
sum(total_value.values())

37.882432253466256

In [78]:
total_value

{(214.0, 210): 2.5997143699910414,
 (215.0, 195): 3.371111556466804,
 (216.0, 204): 1.9693407147629431,
 (217.0, 212): 2.795351099952753,
 (218.0, 207): 1.5985102915396,
 (219.0, 196): 2.416330437474175,
 (220.0, 206): 3.320029642079919,
 (221.0, 199): 1.9248394909857827,
 (222.0, 205): 1.2572050787734272,
 (223.0, 200): 1.4821477408251018,
 (224.0, 208): 1.545301953169824,
 (225.0, 203): 1.561518753225648,
 (226.0, 211): 2.1942004201709273,
 (227.0, 213): 2.1229796933179386,
 (228.0, 209): 1.1747524262592166,
 (229.0, 202): 1.0722500762121299,
 (230.0, 201): 1.9609008834774704,
 (231.0, 197): 4.0935941535953475,
 (232.0, 0): -0.5,
 (233.0, 198): -0.07764652881379225}

In [65]:
preference_table

pid,214.0,215.0,216.0,217.0,218.0,219.0,220.0,221.0,222.0,223.0,224.0,225.0,226.0,227.0,228.0,229.0,230.0,231.0,232.0,233.0
iid,,,,,,,,,,,,,,,,,,,,
194,-1.068539,-0.153286,-3.900802,-0.624137,-1.337434,-1.555706,-2.456722,1.386467,-2.701140,-0.724665,1.162350,-1.063656,-0.818385,0.141493,-1.310275,0.876847,-3.659564,-0.583510,-2.571012,-0.793311
195,1.102737,3.371112,-0.884379,-0.005956,0.852179,1.698784,1.327963,0.082454,-2.675581,-0.415602,0.805990,0.756312,-0.208558,0.214174,-0.440380,1.727921,-1.081816,2.438411,-1.058389,-1.010665
196,0.425077,-0.024999,0.067964,2.390790,-2.569793,2.416330,-0.430509,-1.138999,-0.714973,-2.824994,-2.095204,-1.535706,-0.577508,-1.319036,-0.770364,-1.378013,0.578829,-0.560179,-1.943668,-0.929430
197,-0.117153,0.386421,-1.974546,1.735446,-2.361378,-0.683428,-0.222093,-0.215154,0.464789,-0.846320,1.364342,0.083807,1.766075,0.831786,-0.740999,-1.619782,-0.591009,4.093594,4.291342,-1.128271
198,-2.702477,-0.080764,0.466988,-2.062432,-1.429541,-2.479969,-2.436952,-3.292843,-2.914698,-2.091984,-1.152756,-0.868979,-0.336925,-0.221259,0.901657,0.141870,0.895775,-1.452520,-3.974221,-0.077647
199,0.209847,0.953012,-1.171414,1.449705,0.349651,1.268819,-1.925140,1.924839,-0.150864,1.797146,-0.537381,0.229895,1.697383,2.147479,-1.412813,-1.627216,-1.137689,-0.338865,-1.709087,-0.404938
200,1.322564,1.826139,-1.731384,2.103101,0.686620,1.588445,1.217624,0.499702,0.111163,1.482148,1.646020,2.087964,-0.047579,0.892827,-2.598531,-1.101196,-2.462683,1.020403,1.218151,0.000000
201,0.532173,0.930395,1.143043,0.019842,-2.334316,-0.582343,-0.128726,0.537795,-3.589347,0.638418,0.101136,-0.574341,1.268599,0.036356,-0.266901,0.883574,1.960901,-0.499185,-0.776808,0.173540
202,2.186364,0.318804,0.384456,1.420269,-0.099055,-0.269363,-1.471872,0.362016,-1.347976,-0.509291,0.206210,-1.061587,0.624887,-0.730663,-2.823764,1.072250,-3.056549,-0.421991,-0.463057,0.874820


In [46]:
matched_partners = wave9[wave9['gender']==0].merge(wave9[wave9['gender']==1], left_on=['pid', 'iid'], right_on=['iid', 'pid'], suffixes=('_0', '_1'))

In [53]:
matched_partners.to_csv('matched_partners_wave_9.csv')

In [114]:
df['date_only'] = df['timestamp'].str.split(' ').str[0]

In [118]:
oneday = df[df['date_only']=='2019-11-14']

In [123]:
#bag of words 
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
total_cars_matrix = cv.fit_transform(oneday['cars_list'])

In [127]:
#turn into dataframe

total_cars_df = pd.DataFrame(total_cars_matrix.toarray(), columns=cv.get_feature_names_out())

In [145]:
combodf = pd.concat([oneday.reset_index(drop=True), total_cars_df], axis=1)

In [151]:
combodf

,latitude,longitude,total_cars,cars_list,timestamp,date_only,10,100,101,102,...,90,91,92,93,94,95,96,97,98,99
0,32.091010,34.786330,1,[197],2019-11-14 00:03:03 UTC,2019-11-14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,32.124610,34.830230,1,[116],2019-11-14 00:03:03 UTC,2019-11-14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,32.097650,34.800800,1,[103],2019-11-14 00:03:03 UTC,2019-11-14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,32.078621,34.797969,1,[203],2019-11-14 00:03:03 UTC,2019-11-14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,32.076318,34.767295,1,[248],2019-11-14 00:03:03 UTC,2019-11-14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160353,32.079825,34.774450,2,"[148, 67]",2019-11-14 21:33:02 UTC,2019-11-14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
160354,32.143248,34.792340,3,"[94, 139, 272]",2019-11-14 21:33:02 UTC,2019-11-14,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
160355,32.064615,34.795787,3,"[259, 72, 153]",2019-11-14 21:33:02 UTC,2019-11-14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
160356,32.108610,34.797300,1,[55],2019-11-14 21:33:02 UTC,2019-11-14,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [166]:
groupedup = oneday.sort_values('timestamp').groupby(['latitude', 'longitude', 'timestamp'])[['total_cars']].mean()

In [174]:
groupedup['rolling'] = groupedup['total_cars'].rolling(2).mean()

In [190]:
grouped = oneday.sort_values('timestamp').groupby(['latitude', 'longitude']).rolling(2, on='timestamp')[['total_cars']].mean()


In [202]:
grouped['cars'] = oneday.sort_values('timestamp').groupby(['latitude', 'longitude', 'timestamp'])[['total_cars']].mean()

In [203]:
grouped[(grouped['total_cars'] != grouped['cars']) & (grouped['total_cars'].notnull())]

total_cars  cars
latitude  longitude timestamp                                
32.034470 34.74685  2019-11-14 14:39:03 UTC         1.5   1.0
32.035393 34.75873  2019-11-14 04:09:02 UTC         0.5   0.0
                    2019-11-14 12:30:03 UTC         0.5   1.0
                    2019-11-14 12:36:02 UTC         0.5   0.0
32.036900 34.76250  2019-11-14 05:03:02 UTC         0.5   1.0
...                                                 ...   ...
32.143248 34.79234  2019-11-14 11:06:02 UTC         3.5   3.0
                    2019-11-14 13:03:02 UTC         3.5   4.0
                    2019-11-14 16:54:09 UTC         3.5   3.0
                    2019-11-14 18:18:03 UTC         3.5   4.0
                    2019-11-14 19:54:02 UTC         3.5   3.0

[1982 rows x 2 columns]

In [225]:
grouped.loc[(grouped['total_cars'] - grouped['cars']) > 0, 'Demand'] = 1

grouped.loc[(grouped['total_cars'] - grouped['cars']) < 0, 'Supply'] = 1

In [227]:
grouped.Demand.sum()

997.0

In [228]:
grouped.Supply.sum()

985.0

In [220]:
oneday[oneday['latitude']==32.072323].sort_values('timestamp')[250:300]

,latitude,longitude,total_cars,cars_list,timestamp,date_only
7119225,32.072323,34.790555,2,"[18, 54]",2019-11-14 12:30:03 UTC,2019-11-14
7082557,32.072323,34.790555,2,"[18, 54]",2019-11-14 12:33:03 UTC,2019-11-14
7085497,32.072323,34.790555,2,"[18, 54]",2019-11-14 12:36:02 UTC,2019-11-14
7082993,32.072323,34.790555,2,"[18, 54]",2019-11-14 12:39:02 UTC,2019-11-14
7092512,32.072323,34.790555,2,"[18, 54]",2019-11-14 12:42:02 UTC,2019-11-14
7115525,32.072323,34.790555,2,"[18, 54]",2019-11-14 12:45:02 UTC,2019-11-14
7026960,32.072323,34.790555,2,"[18, 54]",2019-11-14 12:48:03 UTC,2019-11-14
7118317,32.072323,34.790555,2,"[18, 54]",2019-11-14 12:51:02 UTC,2019-11-14
7124652,32.072323,34.790555,2,"[18, 54]",2019-11-14 12:54:03 UTC,2019-11-14
7122201,32.072323,34.790555,2,"[18, 54]",2019-11-14 12:57:03 UTC,2019-11-14


In [99]:
# turn into a dataframe
total_cars_matrix[:,4].sum()

0

In [86]:
total_cars_df.sum()

0       0
1       0
2       0
3       0
4       0
       ..
256     7
257    26
258    29
259    29
260    27
Length: 261, dtype: int64

In [35]:
# make cars_list a list

df['cars_list'].str.sl

0           [[138]]
1            [[64]]
2           [[120]]
3           [[110]]
4           [[190]]
             ...   
20049193       [[]]
20049194       [[]]
20049195       [[]]
20049196       [[]]
20049197       [[]]
Name: cars_list, Length: 20049198, dtype: object

In [33]:
df[df['cars_list'].apply(lambda x: 71 in x)]

TypeError: 'in <string>' requires string as left operand, not int